<a href="https://colab.research.google.com/github/life-ofpi/2023_AI/blob/main/AI_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family = 'Malgun Gothic')
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [ ]:
train

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
3995,ham,I love to cuddle! I want to hold you in my str...
3996,ham,R u in this continent?
3997,ham,We'll you pay over like &lt;#&gt; yrs so its ...
3998,spam,Bored housewives! Chat n date now! 0871750.77....


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#불용어제거
vectorizer = CountVectorizer(stop_words='english')
#bow 벡터로 변환
X = vectorizer.fit_transform(train['Message'])
#이진분류 처리
y = train['Category'].apply(lambda x: 1 if x == 'spam' else 0)



In [ ]:
#Logistic model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# 훈련 세트와 검증 세트 split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100]}

# GridSearchCV
grid_lr = GridSearchCV(LogisticRegression(), param_grid_lr, cv=5)
grid_lr.fit(X_train, y_train)

# 결과 출력
print("logistic 최적 하이퍼파라미터:", grid_lr.best_params_)


lr_model = LogisticRegression(C=100)
lr_model.fit(X_train, y_train)
y_val_pred = lr_model.predict(X_val)

print("검증 정확도:", accuracy_score(y_val, y_val_pred))


logistic 최적 하이퍼파라미터: {'C': 100}
검증 정확도: 0.985


In [ ]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_tfidf = tfidf_vectorizer.fit_transform(train['Message'])

X_train_tfidf, X_val_tfidf, y_train, y_val = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
#model 훈련
lr_model_tfidf = LogisticRegression(C=100)
lr_model_tfidf.fit(X_train_tfidf, y_train)
y_val_pred_tfidf = lr_model_tfidf.predict(X_val_tfidf)

print("검증 정확도:", accuracy_score(y_val, y_val_pred_tfidf))
# CountVectorizer 정확도가 0.985로 더 높음!

검증 정확도: 0.98


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# KNN의 하이퍼파라미터 그리드
param_grid_knn = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
                     'weights': ['uniform', 'distance']}

# GridSearchCV
grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=5)
grid_knn.fit(X_train, y_train)

# 결과 출력
print("KNN 최적 하이퍼파라미터:", grid_knn.best_params_)



# KNN 모델 초기화 및 훈련
knn_model = KNeighborsClassifier(n_neighbors=1,weights='uniform')
knn_model.fit(X_train, y_train)
y_val_pred_knn = knn_model.predict(X_val)
accuracy_knn = accuracy_score(y_val, y_val_pred_knn)
print(f"KNN 검증 정확도: {accuracy_knn}")


KNN 최적 하이퍼파라미터: {'n_neighbors': 1, 'weights': 'uniform'}
KNN 검증 정확도: 0.95875


In [ ]:
#DT의 하이퍼파라미터 그리드
param_grid_dt = {
    'max_depth': [10, 50, 100, 500],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# GridSearchCV
grid_dt = GridSearchCV(DecisionTreeClassifier(), param_grid_dt, cv=5)
grid_dt.fit(X_train, y_train)

# 결과 출력
print("DT 최적 하이퍼파라미터:", grid_dt.best_params_)


# 결정 트리 모델 초기화 및 훈련
dt_model = DecisionTreeClassifier(max_depth=100,min_samples_leaf=1,min_samples_split=5)
dt_model.fit(X_train, y_train)
y_val_pred_dt = dt_model.predict(X_val)
accuracy_dt = accuracy_score(y_val, y_val_pred_dt)
print(f"결정 트리 검증 정확도: {accuracy_dt}")


DT 최적 하이퍼파라미터: {'max_depth': 100, 'min_samples_leaf': 1, 'min_samples_split': 5}
결정 트리 검증 정확도: 0.97375


In [ ]:
#logistic이 제일 좋으니(0.985) logistic model로 test.csv 예측.
lr_model = LogisticRegression(C=100)
lr_model.fit(X_train, y_train)
y_val_pred = lr_model.predict(X_val)

print("logistic model 정확도:", accuracy_score(y_val, y_val_pred))


X_test = vectorizer.transform(test['Message'])
y_test_pred = lr_model.predict(X_test)

logistic model 정확도: 0.985


In [ ]:
# 'test.csv' 데이터에 예측 결과 추가
test['Category'] = ['spam' if pred == 1 else 'ham' for pred in y_test_pred]
test.to_csv('test_predictions.csv', index=False)